# Klassifikation mit *k*-Nearest-Neighbors
Ziel dieser Übung ist das eigenständige Implementieren des Klassifikations-Algorithmus *k*-Nearest-Neighbors innerhalb des Jupyter-Notebooks. Implementieren sie folgende Variante des *k*-Nearest-Neighbors:
- Alle Attribute sind vor der Benutzung auf den Wertebereich $[0;1]$ zu normieren. Beachten Sie dabei, dass für das "Training" des Klassifikators keine Informationen aus den Testdaten verwendet werden dürfen.
- Als Distanzfunktion nutzen Sie bitte die euklidische Distanz.
- Für das Abstimmen der *k* nächsten Nachbarn soll es vier Varianten geben, die mittels eines Parameters an die Klasse übergeben werden können:
    1. Eine einfache Mehrheitsabstimmung unter den Nachbarn
    2. Jeder Nachbar wird mit dem inversen Quadrat der Distanz gewichtet.
    3. Die Stimmen einer Klasse werden mit dem Inversen ihrer Durchschnittsdistanz gewichtet.
    4. Eine Mehrheitsabstimmung gewichtet nach der Verteilung der Klassen.
   
Sie dürfen die Pakete **collections**, **math** und **numpy** für Ihre Implementierung nutzen. Für die Ausführung der Tests benötigen Sie außerdem **pandas**.

# Aufgaben für 6ECTS
Der *k*-nearest Neighbor Algorithmus verwendet für die Klassifikation verschiedene Parameter: Es muss ein fester Wert für **k** und eine der Entscheidungsstrategien gewählt werden. Doch wie wählt man die Parameter sinnvoll? Eine Möglichkeit liefert die Kreuzvalidierung, welche Sie implementieren sollen:
Die Funktion *train* hat einen Parameter **ks**. Falls dieser nicht *None* ist, soll dieser Parameter genutzt werden, um eine Liste von Möglichen *k* Werten zu übergeben. Sie sollen dann wie folgt vorgehen, um aus dieser Liste von *k* Werten das bestmögliche *k* und die bestmögliche der 4 Strategien zu wählen:
- Teilen Sie die Daten zufällig in 5 gleichgroße Teile auf. Nutzen Sie dafür die vorgegebene Funktion *split*.
- Gehen Sie für jedes Kombination für *k* und jeder Stratege *s* wie folgt vor um die besten Werte zu finden:
- Für jede der 5 Datensatzteile "trainieren" sie auf den üblichen 4 Teilen und klassifizieren die Daten des nicht zum Training verwendeten Teil. Zählen Sie die richtigen Klassifikationen.
- Summieren Sie auf, sodass sie die richtigen Entscheidungen über alle 5 Teile erhalten.
- Nun haben sie für jedes Paar (*k*,*s*) eine Anzahl an richtigen Entscheidungen über die 5 Teile.
- Wählen Sie nun die beste Kombination aus und "trainieren" sie auf den ganzen Datensatz. Speichern Sie außerdem das "beste" Paar (*k*,*s*).
- Wird nun die Funktion *predict* mit *best_combination=True* aufgerufen, so sollen der ermittelte Wert für *k* und die ermittelte Strategie statt die übergebenen Werte genutzt werden.


Damit die Aufgabe als sinnvoll bearbeitet gilt, sind folgende Anforderungen zu erfüllen:
- Bei einem Durchlauf durch den Iris-Datensatz sollen keine Ausführungsfehler bestehen und (sehr) gute Werte für die Accuracy geliefert werden.
- Abgabe der Übung bis 26.01.2024 23:59 Uhr im Moodle-Kurs. 

In [51]:

import random  # Just needed for 6ECTS
from collections import Counter

import numpy as np
import pandas as pd


def euclidean_dist(x, y):
    """Calculate the euclidian distance."""
    return np.linalg.norm(x - y)


def split(data, labels, n=5):
    """Split the training points and labels into 5 equal sized parts. Just needed for 6 ECTS."""
    m = list(range(len(data)))
    random.shuffle(m)
    data = np.array(data)
    labels = np.array(labels)
    indices = [np.array(m[i::n]) for i in range(n)]
    data = [data[i].tolist() for i in indices]
    labels = [labels[i].tolist() for i in indices]
    return data, labels


class KNN:

    def __init__(self, dist_fun=euclidean_dist):
        self.dist_fun = dist_fun
        self.trainData = []
        self.trainLabels = []
        self.classifier = {}
        self.k = 0
        self.strategy = ""
        self.strategies = [
            "majority",
            "inverse_squared_distance",
            "inverse_avg_distance",
            "distribution",
        ]

    def train(self, data, labels, ks=None):
        """Train this classifier. Takes a list of samples data and a list of class-labels labels.
        Each sample is a list of numeric values. Each label is a string.
        The parameter ks ist just needed for 6ECTS.
        """
        assert len(data) == len(labels)
        data = self.normalize(data)

        self.trainData = data
        self.trainLabels = labels

        # Find optimal parameters
        if ks:
            bestCombination = (0, "", 0) # k, strategy, quality
            data, labels = split(data, labels)
            i = random.randint(0, 4)
            self.trainData = np.concatenate([data[j] for j in range(5) if j != i])
            self.trainLabels = np.concatenate([labels[j] for j in range(5) if j != i])
            testData = data[i]
            testLabels = labels[i]

            for k in ks:
                for s in self.strategies:
                    predLabels = self.predict(testData, k, s)
                    countDiff = sum(x == y for x, y in zip(predLabels, testLabels))
                    bestCombination = (k, s, countDiff) if countDiff > bestCombination[2] else bestCombination

            self.k, self.strategy = bestCombination[0], bestCombination[1]

    def predict(self, data, k=3, strategy="majority", best_combination=False):
        """Takes a list of samples data. Returns a list of predicted labels for the samples.
        The parameter best_combination ist just needed for 6ECTS.
        """
        data = self.normalize(data)
        if best_combination:
            k, strategy = self.k, self.strategy
        return [self.predict_sample(data, k, strategy) for data in data]

    def predict_sample(self, data, k=3, strategy="majority"):
        """Predicts the label for a single sample data."""
        distances = [euclidean_dist(data, sample) for sample in self.trainData]
        nearestIndices = np.argsort(distances)[:k]
        nearest = {distances[i]: self.trainData[i] for i in nearestIndices}  #distances:vector
        nearestLabels = [self.trainLabels[i] for i in nearestIndices]
        res = Counter({label: 0 for label in set(self.trainLabels)})

        match strategy:

            # classic k-nn
            case "majority":

                return Counter(nearestLabels).most_common()[0][0]

            # weighted by the inverse square of each neighbor
            case "inverse_squared_distance":

                for i, k in enumerate(nearest):
                    res[nearestLabels[i]] += (1 / (k ** 2))
                return res.most_common()[0][0]

            # weighted by the inverse average distance of each class
            case "inverse_avg_distance":

                weights = {label: 0 for label in set(self.trainLabels)}
                count = {label: 0 for label in set(self.trainLabels)}

                for i in range(len(self.trainLabels)):
                    weights[self.trainLabels[i]] += distances[i]
                    count[self.trainLabels[i]] += 1
                for key in weights.keys():
                    weights[key] = 1 / (weights[key] / count[key])
                for i, k in enumerate(nearest):
                    res[nearestLabels[i]] += weights[nearestLabels[i]]
                return res.most_common()[0][0]

            # weighted by distribution of classes
            case "distribution":

                weights = Counter(self.trainLabels)

                for i, k in enumerate(nearest):
                    res[nearestLabels[i]] += (weights[nearestLabels[i]] / len(self.trainLabels))
                return res.most_common()[0][0]

    def normalize(self, data):
        """Normalize data of each attribute"""
        data = np.array(data)
        return data / data.max(axis=0)

# Evaluierung des Klassifikators
Mit diesem Code können Sie Ihre Implementierung anhand des mitgelieferten IRIS-Datensatzes testen. Probieren Sie auch verschiedene (sinnvolle) Werte für den Parameter *k*. Bitte ansonsten in diesem Teil nichts mehr ändern.

In [52]:
def accuracy(predictions, targets):
    """Calculates the accuracy for the given class predictions and true classes."""
    assert len(predictions) == len(targets)
    n_correct = len([p for p, t in zip(predictions, targets) if p == t])
    return n_correct / len(predictions)

In [53]:
def confusion_matrix(predictions, targets):
    """Returns a tuple (labels, m) where m is the confusion matrix and
    labels is the list of matrix rows/columns in same order as in the matrix.
    Rows in the confusion matrix indicate the true target label
    whereas the columns indicate the predicted label of samples.
    """
    assert len(predictions) == len(targets)

    # Map each label to an index.
    unique_vals = list(set(predictions).union(targets))
    mapping = {label: index for index, label in enumerate(unique_vals)}

    # Build and fill the confusion matrixdata.
    m = [[0] * len(mapping) for _ in range(len(mapping))]
    for p, t in zip(predictions, targets):
        row, col = mapping[t], mapping[p]
        m[row][col] += 1
    return unique_vals, m

In [54]:
# Load the csv and drop duplicate entries.
data = pd.read_csv("iris.csv").drop_duplicates()

# Draw a random sample without replacement for the test data.
test_data = data.sample(n=50)

# The other samples are used as training data.
train_data = data.loc[data.index.drop(test_data.index), :]


def df_to_vectors(df):
    """Take a pandas data-frame from the iris dataset as input.
    Returns a tuple (data, labels) where labels is a list of class labels
    and data is the list of sample-vectors
    with each vector represented as a list of numeric values.
    """
    df = df.copy()
    classes = df["species"]
    del df["species"]
    return df.values.tolist(), classes.values.tolist()


# Convert train and test-data to lists of vectors and class labels.
data_train, labels_train = df_to_vectors(train_data)
data_test, labels_test = df_to_vectors(test_data)

In [55]:
clf = KNN()
clf.train(data_train, labels_train)
for strategy in clf.strategies:
    predictions = clf.predict(data_test, strategy=strategy, k=3)
    print("Accuracy of strategy {}: {}".format(
        strategy, accuracy(predictions, labels_test)))
    labels, matrix = confusion_matrix(predictions, labels_test)
    print("Confusion matrix:")
    print("\n".join([str(row) for row in matrix]))
    print("----------")

Accuracy of strategy majority: 0.96
Confusion matrix:
[13, 0, 0]
[2, 17, 0]
[0, 0, 18]
----------
Accuracy of strategy inverse_squared_distance: 0.94
Confusion matrix:
[13, 0, 0]
[3, 16, 0]
[0, 0, 18]
----------
Accuracy of strategy inverse_avg_distance: 0.96
Confusion matrix:
[13, 0, 0]
[2, 17, 0]
[0, 0, 18]
----------
Accuracy of strategy distribution: 0.96
Confusion matrix:
[13, 0, 0]
[2, 17, 0]
[0, 0, 18]
----------


# Delete the following Lines if you just need 3 ECTS!

In [56]:
clf = KNN()
clf.train(data_train, labels_train, ks=[1, 3, 5, 7])
print("Best combination:")
print("k:", clf.k)
print("strategy:", clf.strategy)
print("-----------")
predictions = clf.predict(data_test, best_combination=True)
labels, matrix = confusion_matrix(predictions, labels_test)
print("Accuracy {}".format(accuracy(predictions, labels_test)))
print("Confusion matrix:")
print("\n".join([str(row) for row in matrix]))

Best combination:
k: 1
strategy: majority
-----------
Accuracy 0.96
Confusion matrix:
[13, 0, 0]
[2, 17, 0]
[0, 0, 18]
